In [73]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
# from Model_Build.Functions import *

In [66]:
file_name = 'In_Season/Data/opening_day_rosters.pickle'
with open(file_name, 'rb') as f:
    opening_day_rosters = pickle.load(f)

In [5]:
file_name = 'Model_Build/Data/predict_vorp_regression.pickle'
with open(file_name, 'rb') as f:
    model = pickle.load(f)

In [105]:
def name_adjustment(x):
    x = x.split(' ')
    x = x[0] + ' ' + x[1]      
    return x
file_name = 'In_Season/Data/vorp_predictive_data.csv'
prior_year_predictors = pd.read_csv(file_name, index_col = 0)
prior_year_predictors['Player'] = prior_year_predictors.Player.apply(unidecode.unidecode)
prior_year_predictors['Player'] = prior_year_predictors.Player.apply(name_adjustment)
prior_year_predictors['Player'] = prior_year_predictors.Player.str.replace('.', '')
prior_year_predictors['Player'] = prior_year_predictors.Player.str.replace("'", '')
prior_year_predictors['Player'] = prior_year_predictors.Player.str.lower()

In [106]:
missed_players = list()
vorps = pd.DataFrame(columns = ['Team', 'VORP_projection'])
for key, value in opening_day_rosters.items():
    team = key
    roster = value
    team_vorp = 0
    for index, row in roster.iterrows():
        player = row.Player
        service = row.YOS
        if service > 0:
            player = player.replace('.', '')
            player = player.replace(',', '')
            player = player.replace("'", '')
            player = player.split(' ')
            player = player[0] + ' ' + player[1]
            player = player.lower()
            
            predictors = prior_year_predictors[prior_year_predictors.Player == player]
            if len(predictors) > 0:
                predictors = predictors[['Age', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
                       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
                       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP_Prior_Year']]
                vorp_prediction = model.predict(predictors)[0]
                team_vorp += vorp_prediction
                
            if len(predictors) == 0:
                
                missed_players.append(team + ': ' + player)
            
    series = pd.Series([team, team_vorp], index = vorps.columns)
    vorps = vorps.append(series, ignore_index = True)


In [107]:
missed_players

['Boston Celtics: enes kanter',
 'Brooklyn Nets: nicolas claxton',
 'Golden State Warriors: klay thompson',
 'Houston Rockets: kj martin',
 'Los Angeles Clippers: jayden scrubb',
 'Miami Heat: omer yurtseven',
 'New Orleans Pelicans: nicolas claxton',
 'New York Knicks: wayne selden',
 'Oklahoma City Thunder: mohamed bamba',
 'Oklahoma City Thunder: jonathan isaac',
 'Oklahoma City Thunder: moe wagner',
 'Sacramento Kings: zach collins',
 'Toronto Raptors: ogugua anunoby',
 'Toronto Raptors: sam dekker',
 'Washington Wizards: raulzinho neto']

In [93]:
prior_year_predictors[-100:-50]

,Player,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP_Prior_Year
440,Robert Williams,23,59.222222,1121.805556,25.700000,0.719000,0.008000,0.283000,14.900000,25.600000,...,15.200000,15.000000,3.872222,2.163889,6.036111,0.258000,3.302778,3.530556,6.833333,2.277778
441,Robert Woodard,21,14.805556,51.250000,12.500000,0.425000,0.300000,0.400000,9.700000,30.000000,...,4.100000,23.400000,0.000000,0.000000,0.000000,-0.011000,-3.075000,-2.961111,-6.036111,0.000000
442,Robin Lopez,32,80.861111,1542.055556,16.700000,0.661000,0.042000,0.322000,11.100000,10.200000,...,13.400000,17.100000,3.188889,1.025000,4.213889,0.131000,0.227778,-1.366667,-1.138889,0.455556
443,Rodions Kurucs,22,47.833333,284.722222,8.175000,0.525250,0.634250,0.117500,4.150000,16.600000,...,22.000000,14.375000,-0.683333,0.455556,-0.341667,0.016000,-20.727778,8.200000,-12.527778,-0.227778
444,Rodney Hood,28,125.277778,2145.666667,6.033333,0.462667,0.448333,0.146333,2.566667,10.300000,...,10.600000,14.133333,-1.594444,0.683333,-0.911111,-0.008000,-15.716667,-4.897222,-20.613889,-2.391667
445,Rodney McGruder,29,18.222222,220.944444,16.100000,0.619000,0.343000,0.114000,4.500000,8.100000,...,8.700000,18.100000,0.341667,0.227778,0.569444,0.121000,0.797222,0.000000,0.797222,0.113889
446,Romeo Langford,21,20.500000,322.305556,5.100000,0.436000,0.305000,0.203000,4.700000,9.100000,...,13.500000,11.300000,-0.227778,0.227778,0.000000,0.003000,-5.580556,0.227778,-5.238889,-0.227778
447,Rondae Hollis-Jefferson,26,12.527778,121.861111,14.400000,0.539000,0.056000,0.889000,10.700000,15.400000,...,7.400000,10.900000,0.341667,0.113889,0.341667,0.147000,-1.252778,1.366667,0.113889,0.113889
448,Royce O'Neale,27,80.861111,2552.250000,9.900000,0.599000,0.706000,0.117000,4.400000,18.000000,...,16.700000,9.600000,2.619444,3.758333,6.377778,0.121000,-1.366667,2.391667,1.138889,1.936111
449,Rudy Gay,34,71.750000,1546.611111,14.700000,0.532000,0.443000,0.162000,3.200000,20.900000,...,8.800000,23.600000,0.000000,1.936111,1.936111,0.059000,-0.113889,0.000000,-0.113889,0.683333


In [65]:
i = 2
team = 'Brooklyn-Nets'
link = f'https://basketball.realgm.com/nba/teams/{team}/{str(i+1)}/Rosters/Opening_Day'
tables = pd.read_html(link)
tables[-10]

,Player,Pos,HT,WT,Birth Date,Age,Pre-Draft Team,YOS,Draft Status
0,LaMelo Ball,SG,6-7,185,"Aug 22, 2001",20,Illawarra (Australia),1,"2020 1st Rnd, #3, Charlotte Hornets"
1,James Bouknight,SG,6-4,185,"Sep 18, 2000",21,Connecticut,0,"2021 1st Rnd, #11, Charlotte Hornets"
2,Miles Bridges,SF,6-7,225,"Mar 21, 1998",23,Michigan State,3,"2018 1st Rnd, #12, Los Angeles Clippers"
3,"Vernon Carey, Jr.",PF,6-9,258,"Feb 25, 2001",20,Duke,1,"2020 2nd Rnd, #32, Charlotte Hornets"
4,Gordon Hayward,SF,6-7,230,"Mar 23, 1990",31,Butler,11,"2010 1st Rnd, #9, Utah Jazz"
5,Kai Jones,PF,6-10,225,"Jan 19, 2001",20,Texas,0,"2021 1st Rnd, #19, New York Knicks"
6,Arnoldas Kulboka,SF,6-10,225,"Jan 4, 1998",23,Orlandina Basket (Italy),0,"2018 2nd Rnd, #55, Charlotte Hornets"
7,Scottie Lewis,SF,6-4,190,"Mar 12, 2000",21,Florida,0,"2021 2nd Rnd, #56, Charlotte Hornets"
8,Cody Martin,SF,6-6,200,"Sep 28, 1995",26,Nevada,2,"2019 2nd Rnd, #36, Charlotte Hornets"
9,Jalen McDaniels,PF,6-9,212,"Jan 31, 1998",23,San Diego State,2,"2019 2nd Rnd, #52, Charlotte Hornets"
